# Using a 'Master File' of Variants On Multiple Specimens, Determine If Any Specimens Seem Contaminated

```markdown
## Open in Google Colab

To open this notebook directly in Google Colab, click the badge below:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/your-notebook.ipynb)
```

This code imports several Python libraries commonly used for data analysis and visualization:

* **`pandas`**:  A powerful library for data manipulation and analysis.  It provides data structures like DataFrames, which are essentially tables, making it easy to work with structured data.  Think of it as a more powerful and flexible version of spreadsheets.

* **`glob`**:  This module provides functions for finding all the pathnames matching a specified pattern according to the rules used by the Unix shell.  It's useful for working with multiple files in a directory. For example, you could use `glob` to find all CSV files in a directory.

* **`os`**: This module provides functions for interacting with the operating system, such as navigating directories, creating files, and getting file information.


* **`matplotlib.pyplot`**:  A collection of functions that make matplotlib work like MATLAB. Each pyplot function makes some change to a figure: e.g., creates a figure, creates a plotting area in a figure, plots some lines in a plotting area, decorates the plot with labels, etc.


* **`matplotlib.backends.backend_pdf`**: This part of matplotlib allows you to create PDF files that contain multiple plots or figures.  This is very useful for generating reports or documents that include visualizations.


In [ ]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

1. **`combined_df = pd.read_csv('path to MasterVariantTableXtreme.txt', sep='\t', header=None, low_memory=False)`**:
   - This line uses the pandas library to read the specified file into a DataFrame.
   - `sep='\t'` indicates that the columns in the file are separated by tabs.
   - `header=None` tells pandas that the file doesn't have a header row (the first row is not column names).
   - `low_memory=False` is used to potentially improve performance when dealing with large files by disabling memory optimization strategies.  This can be necessary if the default memory usage approach runs into problems.

2. **`new_column_names = { ... }`**:
   - This creates a dictionary where keys are the original column numbers (0-8) and values are the new, more descriptive names for those columns.  For example, column 0 will be renamed 'Tumor_Sample_Barcode'.

3. **`combined_df = combined_df.rename(columns=new_column_names)`**:
   - This line uses the `rename` function of the DataFrame to update the column names.  It uses the `new_column_names` dictionary to map the old column numbers to the new column names.

In [ ]:
combined_df = pd.read_csv('path to MasterVariantTableXtreme.txt', sep='\t', header=None, low_memory=False)

new_column_names = {
    0: 'Tumor_Sample_Barcode',
    1: 'Chromosome',
    2: 'Start_Position',
    3: 'ref',
    4: 'alt',
    5: 'DP',
    6: 'Ref_DP',
    7: 'Alt_DP',
    8: 'AF'
}

# Use the rename function to update column names
combined_df = combined_df.rename(columns=new_column_names)


This code filters a Pandas DataFrame (`combined_df`) based on several criteria applied to each unique tumor sample barcode. Let's break down the process step-by-step:

1. **Initialization:**
   - An empty list `filtered_dfs` is created to store the filtered dataframes for each barcode.

2. **Iteration over Unique Barcodes:**
   - The code iterates through each unique value in the 'Tumor_Sample_Barcode' column of `combined_df`.  This ensures that the filtering is applied independently to each sample.

3. **Filtering Data for Each Barcode:**
   - **`barcode_df = combined_df[combined_df['Tumor_Sample_Barcode'] == barcode]`**: For the current barcode, it extracts all rows from `combined_df` where the 'Tumor_Sample_Barcode' matches the barcode being processed in the loop.  This creates a subset DataFrame (`barcode_df`) containing only the data for that specific barcode.

   - **`filtered_barcode_df = barcode_df[...]`**: This line performs the core filtering. It applies three conditions to `barcode_df`:
     - `(barcode_df['AF'] < 20)`: Allele frequency ('AF') must be less than 20.
     - `(barcode_df['AF'] > 2)`: Allele frequency ('AF') must be greater than 2.
     - `(barcode_df['DP'] > 200)`: Depth of coverage ('DP') must be greater than 200.

     The `&` operator combines these conditions, meaning that a row is included in `filtered_barcode_df` only if *all three* conditions are true.

4. **Appending to the Filtered List:**
   - **`if not filtered_barcode_df.empty:`**:  This check is crucial. It verifies that the filtering process actually produced some results for the current barcode. If `filtered_barcode_df` is empty (meaning no rows met all three filter criteria), the code avoids appending an empty DataFrame to the list.  This prevents errors later on.
   - **`filtered_dfs.append(filtered_barcode_df)`**: If `filtered_barcode_df` is not empty, it's added to the `filtered_dfs` list.

5. **Concatenation and Output:**
   - **`if filtered_dfs:`**: After processing all barcodes, this condition checks if the `filtered_dfs` list contains any DataFrames.  If the list is not empty, it means at least one barcode had data that passed the filtering steps.
   - **`final_df = pd.concat(filtered_dfs)`**: All the filtered DataFrames in `filtered_dfs` are combined into a single DataFrame `final_df`.
   - **`print(final_df)`**: The resulting `final_df`, containing all rows that passed the filter criteria across all barcodes, is printed.
   - **`else: print("No data found matching the criteria.")`**: If `filtered_dfs` is empty (no barcodes passed the filtering criteria), this message is printed.



In [ ]:
filtered_dfs = []
for barcode in combined_df['Tumor_Sample_Barcode'].unique():
    # Extract rows for the current barcode
    barcode_df = combined_df[combined_df['Tumor_Sample_Barcode'] == barcode]

    # Apply the filtering conditions
    filtered_barcode_df = barcode_df[
        (barcode_df['AF'] < 20) &
        (barcode_df['AF'] > 2) &
        (barcode_df['DP'] > 200)
    ]

    if not filtered_barcode_df.empty:
      filtered_dfs.append(filtered_barcode_df)

# Concatenate all filtered dataframes
if filtered_dfs:
  final_df = pd.concat(filtered_dfs)
  print(final_df)
else:
  print("No data found matching the criteria.")


Very similar code here as the last cell

In [ ]:
filtered_dfs2 = []
for barcode in combined_df['Tumor_Sample_Barcode'].unique():
    subset = combined_df[combined_df['Tumor_Sample_Barcode'] == barcode]
    filtered_subset = subset[
        (subset['AF'] > 40) & (subset['DP'] > 200)
    ]
    filtered_dfs2.append(filtered_subset)

final_df2 = pd.concat(filtered_dfs2, ignore_index=True)
final_df2



The code snippet you provided performs an inner join of two dataframes (`final_df` and `final_df2`) based on shared 'Chromosome' and 'Start_Position' values, then analyzes the resulting matches.
1. **`merged_df = pd.merge(final_df, final_df2, on=['Chromosome', 'Start_Position'], how='inner')`**:

   - This line uses the `pd.merge()` function from the pandas library to combine `final_df` and `final_df2`.
   - `on=['Chromosome', 'Start_Position']`: Specifies that the merge should be performed based on matching values in the 'Chromosome' and 'Start_Position' columns in both dataframes.  Rows where both 'Chromosome' and 'Start_Position' are the same in both dataframes will be combined.
   - `how='inner'`: This indicates an "inner join."  An inner join only includes rows where the join keys (in this case, 'Chromosome' and 'Start_Position') exist in *both* dataframes. If a 'Chromosome'/'Start_Position' combination exists in one dataframe but not the other, those rows are excluded from the `merged_df`.

2. **`match_counts = merged_df.groupby(['Tumor_Sample_Barcode_x','Tumor_Sample_Barcode_y']).size().sort_values(ascending=False)`**:

   - This line calculates the frequency of matching tumor sample barcodes in the merged dataframe. Here's a step-by-step breakdown:
     - `merged_df.groupby(['Tumor_Sample_Barcode_x','Tumor_Sample_Barcode_y'])`: This groups the `merged_df` based on unique combinations of 'Tumor_Sample_Barcode_x' and 'Tumor_Sample_Barcode_y'.  Notice the '_x' and '_y' suffixes.  Since the merge involved columns with the same name ('Tumor_Sample_Barcode') from both input DataFrames, pandas automatically appends '_x' and '_y' to distinguish the columns.  '_x' refers to the 'Tumor_Sample_Barcode' from `final_df` and '_y' from `final_df2`.
     - `.size()`:  For each unique group (each unique combination of 'Tumor_Sample_Barcode_x' and 'Tumor_Sample_Barcode_y'), this calculates the size of the group, i.e., the number of rows in the `merged_df` that share those particular barcode combinations. This effectively counts how many times each pair of tumor barcodes appears in the merged data.
     - `.sort_values(ascending=False)`: The results are sorted in descending order of count. This places the most frequent pairs of barcodes (the ones with the most matching variants between samples) at the top of the resulting series.

3. **`match_counts`**:

   - This variable now holds a pandas Series where the index is the tuple `('Tumor_Sample_Barcode_x','Tumor_Sample_Barcode_y')` representing pairs of tumor barcodes, and the value is the number of matching variants found between those two samples.


In [ ]:
merged_df = pd.merge(final_df, final_df2, on=['Chromosome', 'Start_Position'], how='inner')
match_counts = merged_df.groupby(['Tumor_Sample_Barcode_x','Tumor_Sample_Barcode_y']).size().sort_values(ascending=False)
match_counts


This code snippet generates a PDF file named `distribution_plots.pdf`, containing bar plots that visualize the distribution of variant counts for each unique tumor sample barcode.

1. **Setup:**
   - `with PdfPages('distribution_plots.pdf') as pdf:`: This line creates a PDF file named `distribution_plots.pdf`.  The `with` statement ensures that the file is properly closed even if errors occur.  All subsequent `pdf.savefig()` calls will add the current figure to this PDF.

2. **Iterating through Barcodes:**
   - `for barcode in match_counts.index.get_level_values(0).unique():`: This loop iterates through each unique value in the first level of the `match_counts` index.  Recall that `match_counts` is a pandas Series where the index is a MultiIndex (a hierarchical index) with two levels: 'Tumor_Sample_Barcode_x' and 'Tumor_Sample_Barcode_y'.  This line gets all the unique values of 'Tumor_Sample_Barcode_x'.  Each unique barcode represents a different tumor sample.


3. **Extracting and Plotting Counts:**
   - `barcode_counts = match_counts[barcode]`: This line extracts the counts associated with the current `barcode` from the `match_counts` series.  Since `match_counts` is indexed by a MultiIndex, selecting by `barcode` will give you the counts for all corresponding `Tumor_Sample_Barcode_y` barcodes.  The resulting `barcode_counts` is a pandas Series.
   - `plt.figure(figsize=(8, 6))`: Creates a new plot figure with a specified size of 8 inches wide and 6 inches high.
   - `barcode_counts.plot(kind='bar')`: Generates a bar plot of the `barcode_counts`.  The x-axis will represent the `Tumor_Sample_Barcode_y` and the y-axis the number of matching variants.
   - `plt.title(...)`, `plt.xlabel(...)`, `plt.ylabel(...)`: These lines set the title and labels for the x and y axes of the plot to make the plot more informative.
   - `plt.xticks(rotation=45, ha='right')`: Rotates the x-axis tick labels by 45 degrees to prevent overlap. `ha='right'` aligns the text to the right of the tick marks.
   - `plt.tight_layout()`: This is important for ensuring that the labels and title of the plot don't overlap with the plot itself, especially when the x-axis labels are long or rotated.
   - `pdf.savefig()`: Saves the current plot (the bar plot) as a page in the `distribution_plots.pdf` file.
   - `plt.close()`: Closes the current figure to free up memory and avoid potential issues when creating many plots. This is essential for large numbers of plots to prevent memory exhaustion.

4. **Completion Message:**
   - `print("Plots saved to distribution_plots.pdf")`:  Prints a message confirming that the plots have been saved to the PDF file.


In [ ]:
# Assuming 'match_counts' is already defined from the previous code

with PdfPages('distribution_plots.pdf') as pdf:
    for barcode in match_counts.index.get_level_values(0).unique():
        barcode_counts = match_counts[barcode]

        plt.figure(figsize=(8, 6))
        barcode_counts.plot(kind='bar')
        plt.title(f"Distribution of Counts for {barcode}")
        plt.xlabel("Tumor_Sample_Barcode_y")
        plt.ylabel("Count")
        plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
        plt.tight_layout()  # Adjust layout to prevent labels from overlapping
        pdf.savefig()
        plt.close() # Close the plot to free up memory

print("Plots saved to distribution_plots.pdf")